In [50]:
import pandas as pd
import re
import datetime
import numpy as np
from nltk import tokenize

In [51]:
msft = pd.read_excel('MicrosoftReviews.xlsx')
intel = pd.read_excel('IntelReviews.xlsx')
cisco = pd.read_excel('CiscoReviews.xlsx')

In [52]:
msft['Company'] = 'Microsoft'
intel['Company'] = 'Intel'
cisco['Company'] = 'Cisco'

In [53]:
data = pd.concat([msft, intel, cisco])
data.reset_index(inplace=True, drop=True)
data

,Headline,Status,Date,Pros,Cons,Company
0,Best Place To Work,"Current Employee, less than 1 year","Apr 14, 2021","Benefits, D&I, work life balance",I am home at Microsoft.,Microsoft
1,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013","1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,Microsoft
2,"low RSUs, salary, bonuses and career growth","Current Employee, less than 1 year","Apr 5, 2021","Great work life balance, good health benefits....","Average Engineers, Scientists and PMs with not...",Microsoft
3,"Slow movement, massive rewards","Current Employee, more than 5 years","Feb 22, 2021",Pay and compensation regularly above market. R...,"Benefits standard for tech, but on the lower r...",Microsoft
4,Supports personal career development,"Former Employee, less than 1 year","Mar 18, 2021",Ample on-line resources for everything from le...,I never thought laptops would be an issue but ...,Microsoft
...,...,...,...,...,...,...
61239,An ambitious company with its heart in the rig...,"Current Employee, less than 1 year","Jun 13, 2008",An ambitious company with its heart in the rig...,"Despite working for a networking company, comm...",Cisco
61240,Treat your front-line customer-facing staff be...,"Current Employee, less than 1 year","Jun 12, 2008",Breadth of technology - fantastic being able t...,Have no life - submitting numbers on Sunday af...,Cisco
61241,Quick,"Former Employee, less than 1 year","May 2, 2008",Better career path._x000D_\nRegional positions...,number of resources needs to increase,Cisco
61242,Hunt for the best company to work for,"Current Employee, less than 1 year","May 1, 2008","Best work place, Best recognition, Best Teams ...",there are no down sides,Cisco


In [54]:
#for i in range(len(data)):
data['Date'] = data['Date'].apply(lambda x: re.sub(r'[^\w\s]', '', x).strip())
data['Date'] = data['Date'].apply(lambda x: np.NaN if str(x) == 'nan' else datetime.datetime.strptime(str(x), "%b %d %Y").year)
#data['Year'] = data['Date'].apply(lambda x: x.year())
data

TypeError: 'str' object is not callable

In [ ]:
data = data.loc[data['Date'] > 2018]
data.reset_index(drop=True, inplace=True)
data

In [ ]:
data['Date'].value_counts()

In [ ]:
keywords = ['disability', 'disabled person', 'disabilities', 'autism', 'accessibility', 'accessible', 'inclusion', 'inclusivity', 
            'inclusive', 'disabled employee', 'impaired', 'captioning', 'voice-to-text', 'voice to text', 'benefits',
            'discriminate', 'discriminating','discrimination', 'equal employment', 'equal opportunity', 'mental', 'accommodation', 
            'accommodating', 'diversity', 'diverse', 'ndeam', 'national disability employment awareness month', 'a11y', 'abilities',
            'paralysed', 'paralysis', 'transcribed', 'transcription', 'speech recognition', 'blind', 
           'blindness', 'culture', 'screen reader', 'representation', 'therapy', 'disorder', 'dei', 'd&i', 'wheelchair', 
            'hearing', 'neurodiverse', 'deafness']

In [ ]:
def keys(str):
    key = ''
    for i in keywords:
        if re.search(r'\b' + i + r'\b', str.lower()):
            key +=  i + ' ' 
    return key 

def isRelevant(str):
    for i in keywords:
        #a = '\b' + i + '\b'
        if re.search(r'\b' + i + r'\b', str.lower()):
            return 1 
    return 0

In [ ]:
relevance, key, labels = [], [], []
for i in range(len(data)):
    relevance.append(isRelevant(data.Pros[i]) or isRelevant(data.Cons[i]))
    key.append(keys(data.Pros[i]) or keys(data.Cons[i]))
    #labels.append(tags(data.Pros[i]) or tags(data.Cons[i]))
data['Relevance'] = relevance
data['Keywords'] = key
#data['Labels'] = labels

In [ ]:
data['Relevance'].value_counts()

In [ ]:
data = data.loc[data['Relevance'] == 1]
data

In [ ]:
data = data.drop(['Relevance'], axis = 1)
data['Rel_Pros'] = data['Pros'].apply(lambda x: isRelevant(x))
data['Rel_Cons'] = data['Cons'].apply(lambda x: isRelevant(x))

In [ ]:
data['Company'].value_counts()

In [ ]:
pros = data.loc[data['Rel_Pros'] == 1]
pros = pros.drop(['Cons', 'Rel_Pros', 'Rel_Cons'], axis = 1)
pros = pros.drop_duplicates(subset=['Pros'])
pros.reset_index(drop=True, inplace=True)
cons = data.loc[data['Rel_Cons'] == 1]
cons = cons.drop(['Pros', 'Rel_Pros', 'Rel_Cons'], axis = 1)
cons = cons.drop_duplicates(subset=['Cons'])
cons.reset_index(drop=True, inplace=True)

In [ ]:
pros

In [ ]:
def RemBullets(string):
    digits = '1234567890'
    sym = '.)-'
    for x in digits:
        for y in sym:
            z = x+y
            if x+y in string:
                string = string.replace(z, '').strip()
    return string

def RelText(string):
    if '\r\n' in string:
        sent = string.split('\r\n')
    else:
        sent = tokenize.sent_tokenize(string)
    res = []
    for i in sent:
        for j in keywords:
            if j in i.lower():
                res.append(i)
    return ' '.join(list(set(res)))

In [ ]:
pros['RelPros'] = pros['Pros'].apply(lambda x: RemBullets(x))
pros['RelPros'] = pros['RelPros'].apply(lambda x: RelText(x))
pros['RelPros'] = pros['RelPros'].apply(lambda x: x[1:].strip() if (x[0] == '-') | (x[0] == '*') | (x[0] == '+') else x)
cons['RelCons'] = cons['Cons'].apply(lambda x: RemBullets(x))
cons['RelCons'] = cons['RelCons'].apply(lambda x: RelText(x))
cons['RelCons'] = cons['RelCons'].apply(lambda x: x[1:].strip() if (x[0] == '-') | (x[0] == '*') | (x[0] == '+') else x)

In [71]:
pros = pd.read_excel('Pros.xlsx')
cons = pd.read_excel('Cons.xlsx')

In [72]:
show = []
for i in range(len(pros)):
    keys = pros.Keywords[i].split()
    str = pros.RelPros[i].lower()
    for i in keys:
        str = str.replace(i, '<b>' + i + '</b>')
    if str[:3] == '<b>':
        str = str[3:].capitalize()
        show.append('<b>' + str)
    else:
        show.append(str.capitalize())
pros['Show'] = show

In [73]:
show = []
for i in range(len(cons)):
    keys = cons.Keywords[i].split()
    str = cons.RelCons[i].lower()
    for i in keys:
        str = str.replace(i, '<b>' + i + '</b>')
    if str[:3] == '<b>':
        str = str[3:].capitalize()
        show.append('<b>' + str)
    else:
        show.append(str.capitalize())
cons['Show'] = show

In [70]:
# str = '<b>benefits</b>, <b>d&i</b>, work life balance'
# str[3:].capitalize()

In [74]:
pros

,Headline,Status,Date,Pros,Company,Keywords,RelPros,Show
0,Best Place To Work,"Current Employee, less than 1 year",2021,"Benefits, D&I, work life balance",Microsoft,benefits d&i,"Benefits, D&I, work life balance","<b>Benefits</b>, <b>d&i</b>, work life balance"
1,"low RSUs, salary, bonuses and career growth","Current Employee, less than 1 year",2021,"Great work life balance, good health benefits....",Microsoft,benefits d&i,"Great work life balance, good health benefits....","Great work life balance, good health <b>benefi..."
2,"Slow movement, massive rewards","Current Employee, more than 5 years",2021,Pay and compensation regularly above market. R...,Microsoft,inclusion benefits diverse culture,"Great benefits, tuition reimbursement especial...","Great <b>benefits</b>, tuition reimbursement e..."
3,Tech company with a moral compass,"Current Employee, less than 1 year",2021,Senior leadership has driven top-down culture ...,Microsoft,inclusion culture,Senior leadership has driven top-down culture ...,Senior leadership has driven top-down <b>cultu...
4,"Great Company, but only for Engineering Organi...","Current Employee, less than 1 year",2021,- Excellent employee culture & groups to explo...,Microsoft,benefits culture,"In the face of the pandemic, our benefits & fl...","In the face of the pandemic, our <b>benefits</..."
...,...,...,...,...,...,...,...,...
5633,Not good for freshers,"Current Employee, less than 1 year",2019,"Good environment , pay, benefits,",Cisco,benefits,"Good environment , pay, benefits,","Good environment , pay, <b>benefits</b>,"
5634,Business manager,"Former Employee, more than 10 years",2019,Good benefits and work culture,Cisco,benefits culture,Good benefits and work culture,Good <b>benefits</b> and work <b>culture</b>
5635,Must work for Cisco atleast once in lifetime f...,"Current Employee, more than 3 years",2019,Dream company for any network engineer. Amazin...,Cisco,culture,"Amazing facilities, good campus, good work cul...","Amazing facilities, good campus, good work <b>..."
5636,Engineering Manager,"Current Employee, more than 5 years",2019,Great environment and culture and flexibility,Cisco,culture,Great environment and culture and flexibility,Great environment and <b>culture</b> and flexi...


In [75]:
cons

,Headline,Status,Date,Cons,Company,Keywords,RelCons,Show
0,"Slow movement, massive rewards","Current Employee, more than 5 years",2021,"Benefits standard for tech, but on the lower r...",Microsoft,inclusion benefits diverse culture,"Benefits standard for tech, but on the lower r...","<b>Benefits</b> standard for tech, but on the ..."
1,"Great Company, but only for Engineering Organi...","Current Employee, less than 1 year",2021,- Unless you are in the engineering organizati...,Microsoft,benefits culture,"Despite positive company culture, not all lead...","Despite positive company <b>culture</b>, not a..."
2,Microsoft,"Current Employee, more than 10 years",2021,"some groups are not as nimble as others, job f...",Microsoft,d&i,"some groups are not as nimble as others, job f...","Some groups are not as nimble as others, job f..."
3,Incredible Company,"Current Employee, less than 1 year",2021,Busy Culture if you want to excel,Microsoft,culture,Busy Culture if you want to excel,Busy <b>culture</b> if you want to excel
4,Great Company and awesome benefits,"Current Employee, more than 1 year",2021,"Pay is not as great as some other places, howe...",Microsoft,culture,"Pay is not as great as some other places, howe...","Pay is not as great as some other places, howe..."
...,...,...,...,...,...,...,...,...
879,Systems Engineering Manager,"Current Employee, more than 10 years",2019,"Lay-off culture, too many reorgs, Business Uni...",Cisco,culture,"Lay-off culture, too many reorgs, Business Uni...","Lay-off <b>culture</b>, too many reorgs, busin..."
880,Cisco Hyperflex India - there are problems here,"Current Employee, more than 1 year",2019,The india team is new and the ppl in the US cl...,Cisco,diverse,Why expand in India if the team doesn't want i...,Why expand in india if the team doesn't want i...
881,Stay away from this company,"Former Employee, more than 3 years",2019,"Arrogant management, big focus on OPEX , the w...",Cisco,benefits culture,Pay is rather low compare to the market_x000D_...,Pay is rather low compare to the market_x000d_...
882,Great Place to work and learn,"Current Employee, less than 1 year",2019,Slow moving Constant changes Culture,Cisco,culture,Slow moving Constant changes Culture,Slow moving constant changes <b>culture</b>


In [77]:
data.to_excel('Relevant_Glassdoor.xlsx', header=True, index=False)
pros.to_excel('Pros.xlsx', header=True, index=False)
cons.to_excel('Cons.xlsx', header=True, index=False)

In [ ]:
pros['Company'].value_counts()

In [ ]:
cons['Company'].value_counts()